In [62]:
!pip install xgboost

     |████████████████████████████████| 157.5 MB 95 kB/s s eta 0:00:01


In [160]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score


In [146]:
df_pitch = pd.read_csv("Q2_pitches_train.csv")
df_pitch_submit = pd.read_csv("Q2_pitches_test.csv")
#Data load and preprocessing
#df_pitch.columns
#Replacing bat_score_before and Field_score because they are correlated with inning with RunDiff (differential)
#Used seaborn heatmap to check the multicollinearity, even though understanding of innings is not really numeric
#correlations = df_pitch.loc[:,~df_pitch.columns.isin(['pitch_type','batterid','pitcherid','cid'])].corr()
#sns.heatmap(correlations)
df_pitch['RunDiff'] = df_pitch['field_score'] - df_pitch['bat_score_before']
del df_pitch['bat_score_before']
del df_pitch['field_score']
df_pitch_submit['RunDiff'] = df_pitch_submit['field_score'] - df_pitch_submit['bat_score_before']
del df_pitch_submit['bat_score_before']
del df_pitch_submit['field_score'] 
del df_pitch_submit['FF'] 
del df_pitch_submit['FT'] 
del df_pitch_submit['CB'] 
del df_pitch_submit['SL'] 
del df_pitch_submit['CH'] 
#Setting up for RandomizedSearchCV to fine tune the XgBoost on the entire train dataset
X= df_pitch.loc[:,~df_pitch.columns.isin(['pitch_type'])]
y= df_pitch["pitch_type"] 

X_submit = df_pitch_submit.loc[:,~df_pitch_submit.columns.isin(['pitch_type'])]


In [149]:
#model_full = XGBClassifier()
#parameters = {"learning_rate": [0.1, 0.01, 0.001],
 #              "max_depth": [4, 6,8,10],
  #             "min_child_weight": [5, 10, 15, 20],
   #            "n_estimators": [100, 250, 500]}
#We could use other parameters as well, but even this ran for some time on my laptop
#xgb_rscv = RandomizedSearchCV(model_full, param_distributions = parameters, scoring = "f1_micro",
                            # cv = 3, verbose = 3, random_state = 40 )
#model_xgboost = xgb_rscv.fit(X, y)
#Printing the best parameters
#print("Learning Rate: ", model_xgboost.best_estimator_.get_params()["learning_rate"])
#print("Max Depth: ", model_xgboost.best_estimator_.get_params()["max_depth"])
#print("min_child_weight: ",model_xgboost.best_estimator_.get_params()["min_child_weight"])
#print("n_estimators: ", model_xgboost.best_estimator_.get_params()["n_estimators"])

In [138]:
#using these parameters to check the accuracy and classification using train and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)
#create xgb instance with the best parameters
clf = XGBClassifier(learning_rate=0.1,max_depth=10,min_child_weight=15,n_estimators=250, objective= 'multi:softprob')
clf.fit(X_train, y_train)

/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:09:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=15, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [141]:
#make predictions on the test set to check accuracy and other metrics
val_preds = clf.predict(X_test)
prob_p= clf.predict_proba(X_test)
#run classification report to see the total precision, recall and f1-score
class_report = classification_report(y_test, val_preds)
print(class_report)
print(clf.classes_)
#Use above classes to convert the probability matrix to dataframe
pred_df_prob=pd.DataFrame(data=prob_p[:,:], columns=['CB', 'CH', 'FF', 'FT', 'SL'])
print(pred_df_prob)

              precision    recall  f1-score   support

          CB       0.45      0.18      0.26     12080
          CH       0.44      0.24      0.31     14513
          FF       0.55      0.79      0.65     48237
          FT       0.56      0.53      0.54     21662
          SL       0.48      0.37      0.42     23737

    accuracy                           0.53    120229
   macro avg       0.50      0.42      0.43    120229
weighted avg       0.51      0.53      0.50    120229

['CB' 'CH' 'FF' 'FT' 'SL']
              CB        CH        FF        FT        SL
0       0.182459  0.043407  0.195112  0.250680  0.328343
1       0.016524  0.068341  0.181756  0.546381  0.186998
2       0.004794  0.018477  0.887169  0.043855  0.045705
3       0.248402  0.109761  0.217486  0.309546  0.114805
4       0.128486  0.013774  0.593309  0.200659  0.063772
...          ...       ...       ...       ...       ...
120224  0.041667  0.045187  0.413044  0.242647  0.257455
120225  0.073791  0.472989  

In [150]:
#Run the same model for the entire training dataset and then predict on the test set which is the submission
clf.fit(X, y)

/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[03:15:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=15, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [152]:
#make predictions on the final test set to give out the probabilities for each pitch to submit.
submit_preds = clf.predict(X_submit)
submit_prob_p= clf.predict_proba(X_submit)
#run classification report to see the total precision, recall and f1-score
print(clf.classes_)
#Use above classes to convert the probability matrix to dataframe
pred_submit_prob=pd.DataFrame(data=submit_prob_p[:,:], columns=['CB', 'CH', 'FF', 'FT', 'SL'])
print(pred_submit_prob)

['CB' 'CH' 'FF' 'FT' 'SL']
              CB        CH        FF        FT        SL
0       0.001054  0.009531  0.798821  0.132513  0.058082
1       0.002250  0.082494  0.171777  0.033153  0.710326
2       0.001685  0.041127  0.401256  0.046130  0.509803
3       0.003688  0.053667  0.438138  0.025201  0.479306
4       0.004332  0.036696  0.522732  0.020081  0.416160
...          ...       ...       ...       ...       ...
160301  0.159269  0.172834  0.582920  0.037037  0.047940
160302  0.193258  0.038255  0.559172  0.086524  0.122791
160303  0.061501  0.061146  0.786870  0.054594  0.035888
160304  0.061501  0.061146  0.786870  0.054594  0.035888
160305  0.061501  0.061146  0.786870  0.054594  0.035888

[160306 rows x 5 columns]


In [157]:
#Merge the probability with the previous data.
df_pitch_submit_data=df_pitch_submit.merge(pred_submit_prob, left_index=True, right_index=True)

In [159]:
#Export
df_pitch_submit_data.to_csv("Q2_pitches_test_probabilities.csv", index=False)